In [158]:
L = 2;

In [159]:
using Random
using LinearAlgebra
using SparseArrays
using DelimitedFiles
file = raw"Grover_gates_data.txt"
M = readdlm(file)
Gates_data_1 = M[:,1];
Gates_data_2 = M[:,2];
Gates_data_3 = M[:,3];

In [160]:
#DELTA = 0.01

In [161]:
using DelimitedFiles
file = raw"Noise_data.txt"
M = readdlm(file)
NOISE = M[:,1];

In [162]:
Number_of_Gates = 2*(2*L^2-6*L+5)+2*L+4*L-4

10

In [163]:
length(NOISE)

10

In [164]:
I2 = [1 0; 0 1];
Z = [1 0;0 -1];
H = (1/sqrt(2))*[1 1;1 -1]
Rx(theta)=exp(-1im*(theta/2)*[1 1;1 1]);
Hadamard(noise) = exp(-1im*(pi/2+noise)*(I2-H)) #Ry(pi/2+noise)*Pauli_Z;
CX(noise) = exp(-1im*((pi/2+noise))*[1 1;1 1]);
Identity(dimension) = 1* Matrix(I, dimension, dimension);
int(x) = floor(Int,x);

In [165]:
Hadamard(0.01)

2×2 Matrix{ComplexF64}:
 0.707136+0.00292874im   0.707036-0.0070706im
 0.707036-0.0070706im   -0.706936+0.0170699im

In [166]:
function Matrix_Gate(Gate, Qubit) # Previously known as multi qubit gate.
    
    ## The case Qubit=1 is treated differently because we need to
    # initialize the matrix as U before starting the kronecker product.
    
    if Qubit == 1
        
        M = sparse(Gate)
        for i=2:L
            M = kron(M, sparse([1 0;0 1]))
        end
        
    else
        
        M = sparse([1 0;0 1])
        for i=2:L
            if i == Qubit
                M = kron(M, Gate)
            else
                M = kron(M, sparse([1 0;0 1]))
            end
        end
    end
    
    return M
end;

In [167]:
function CU(U,c,t)
    
    I2 = sparse([1 0;0 1])
    Z = sparse([1 0;0 -1])

    PI_0 = (I2+Z)/2
    PI_1 = (I2-Z)/2
     
    #function Rx(Noise)
        #A = cos((pi+Noise)/2)
        #B = -1im*sin((pi+Noise)/2)
        #return 1im*[A B;B A]
    #end
    
    Matrices = Dict("I" => I2,"PI_0" => PI_0,"U" => U, "PI_1" => PI_1)
    
    p0 = fill("I", L)
    p1 = fill("I", L)
    
    p0[c] = "PI_0"
    p1[c] = "PI_1"
    p1[t] = "U"

    
    PI_0_matrix = Matrices[p0[1]]
    for i = 2:L
        PI_0_matrix = kron(PI_0_matrix,Matrices[p0[i]])
    end        
        
    PI_1_matrix = Matrices[p1[1]]   
    for i = 2:L
        PI_1_matrix = kron(PI_1_matrix,Matrices[p1[i]])        
    end
           
    #return p0,p1
    return PI_0_matrix + PI_1_matrix     
end;

In [168]:
using PyCall
py"""
import numpy
import numpy.linalg
def adjoint(psi):
    return psi.conjugate().transpose()
def psi_to_rho(psi):
    return numpy.outer(psi,psi.conjugate())
def exp_val(psi, op):
    return numpy.real(numpy.dot(adjoint(psi),op.dot(psi)))
def norm_sq(psi):
    return numpy.real(numpy.dot(adjoint(psi),psi))
def normalize(psi,tol=1e-9):
    ns=norm_sq(psi)**0.5
    if ns < tol:
        raise ValueError
    return psi/ns
def is_herm(M,tol=1e-9):
    if M.shape[0]!=M.shape[1]:
        return False
    diff=M-adjoint(M)
    return max(numpy.abs(diff.flatten())) < tol
def is_unitary(M,tol=1e-9):
    if M.shape[0]!=M.shape[1]:
        return False
    diff=M.dot(adjoint(M))-numpy.identity((M.shape[0]))
    return max(numpy.abs(diff.flatten())) < tol
def eigu(U,tol=1e-9):
    (E_1,V_1)=numpy.linalg.eigh(U+adjoint(U))
    U_1=adjoint(V_1).dot(U).dot(V_1)
    H_1=adjoint(V_1).dot(U+adjoint(U)).dot(V_1)
    non_diag_lst=[]
    j=0
    while j < U_1.shape[0]:
        k=0
        while k < U_1.shape[0]:
            if j!=k and abs(U_1[j,k]) > tol:
                if j not in non_diag_lst:
                    non_diag_lst.append(j)
                if k not in non_diag_lst:
                    non_diag_lst.append(k)
            k+=1
        j+=1
    if len(non_diag_lst) > 0:
        non_diag_lst=numpy.sort(numpy.array(non_diag_lst))
        U_1_cut=U_1[non_diag_lst,:][:,non_diag_lst]
        (E_2_cut,V_2_cut)=numpy.linalg.eigh(1.j*(U_1_cut-adjoint(U_1_cut)))
        V_2=numpy.identity((U.shape[0]),dtype=V_2_cut.dtype)
        for j in range(len(non_diag_lst)):
            V_2[non_diag_lst[j],non_diag_lst]=V_2_cut[j,:]
        V_1=V_1.dot(V_2)
        U_1=adjoint(V_2).dot(U_1).dot(V_2)
    # Sort by phase
    U_1=numpy.diag(U_1)
    inds=numpy.argsort(numpy.imag(numpy.log(U_1)))
    return (U_1[inds],V_1[:,inds]) # = (U_d,V) s.t. U=V*U_d*V^\dagger
"""

py"""
f = open('exact_G_eigenvalues'+'.txt', 'w')
def Write_file(Eigenvalue):
    f = open('exact_G_eigenvalues'+'.txt', 'a')
    f.write(str(Eigenvalue) +'\n')
"""

In [169]:
U_0 = [-1 0 0 0; 0 1 0 0; 0 0 1 0;0 0 0 1];
A = ones(2^L,2^L);
U_x = (2/2^L)*A-Identity(2^L); # 2\s><s|-I
G_exact = U_x*U_0
V = py"eigu"(G_exact)[2]

4×4 Matrix{ComplexF64}:
 0.707107+0.0im       …          0.0+0.0im        0.0+0.0im
      0.0-0.408248im        0.710106+0.0im   0.403009+0.0im
      0.0-0.408248im     -0.00603653+0.0im  -0.816474+0.0im
      0.0-0.408248im       -0.704069+0.0im   0.413465+0.0im

### Grover matrix constructed from data file

In [170]:
#DELTA = 0.01
function Eigenvalues(DELTA)
    
    U_list = [];
    U_noise_list = [];
    U_x_delta = sparse(Identity(2^L));
    #ux_list = []
    NOISE_list = []

    Gates_data_new_1 = []
    Gates_data_new_2 = []
    Gates_data_new_3 = []
    
    # U_x
    for i = (2*L^2-4*L+5)+1 : 2*(2*L^2-6*L+5)+2*L+4*L-4
        if Gates_data_1[i] == "H"
            
            
            epsilon = NOISE[i]
            push!(NOISE_list,epsilon)
            h_matrix = Matrix_Gate(Hadamard(DELTA*epsilon), Gates_data_3[i])
            U_x_delta *= h_matrix
        
            push!(Gates_data_new_1,"H")
            push!(Gates_data_new_2,0.0)
            push!(Gates_data_new_3,Gates_data_3[i])
        
            push!(U_noise_list,h_matrix) # Noise.
        
            push!(U_list,Matrix_Gate(Hadamard(0.0), Gates_data_3[i])) # Noiseless.
            
        elseif Gates_data_1[i] == "X"
        
            epsilon = NOISE[i]
            push!(NOISE_list,epsilon)        
            x_matrix = Matrix_Gate(CX(DELTA*epsilon),Gates_data_3[i])
            U_x_delta *= x_matrix
        
            push!(Gates_data_new_1,"X")
            push!(Gates_data_new_2,0.0)
            push!(Gates_data_new_3,Gates_data_3[i]) 
        
            push!(U_noise_list,x_matrix) # Noise.
        
            push!(U_list,Matrix_Gate(CX(0.0),Gates_data_3[i])) # Noiseless.
            
        else
            #push!(ux_list,"CRX")
        
            epsilon = NOISE[i]
            push!(NOISE_list,epsilon)        
            rx_matrix = CU(Rx(Gates_data_1[i]+DELTA*epsilon), Gates_data_2[i], Gates_data_3[i])
            U_x_delta *= rx_matrix
        
            push!(Gates_data_new_1,Gates_data_1[i])
            push!(Gates_data_new_2,Gates_data_2[i])
            push!(Gates_data_new_3,Gates_data_3[i])
        
            push!(U_noise_list,rx_matrix) # Noise.
        
            push!(U_list,CU(Rx(Gates_data_1[i]), Gates_data_2[i], Gates_data_3[i])) # Noiselss.
            
        end
    end
    
    U_0_delta = sparse(Identity(2^L));
    
    #u0_list = []
    # U_0
    for i = 1 : 2*L^2-4*L+5
        if Gates_data_1[i] == "H"
        
            epsilon = NOISE[i]
            push!(NOISE_list,epsilon)        
            h_matrix = Matrix_Gate(Hadamard(DELTA*epsilon), Gates_data_3[i])
            U_0_delta *= h_matrix
        
            push!(Gates_data_new_1,"H")
            push!(Gates_data_new_2,0.0)
            push!(Gates_data_new_3,Gates_data_3[i])
        
            push!(U_noise_list,h_matrix) # Noise.
        
            push!(U_list,Matrix_Gate(Hadamard(0.0), Gates_data_3[i])) # Noiseless.
            
        elseif Gates_data_1[i] == "X"

        
            epsilon = NOISE[i]
            push!(NOISE_list,epsilon)        
            x_matrix = Matrix_Gate(CX(DELTA*epsilon),Gates_data_3[i])
            U_0_delta *= x_matrix
        
            push!(Gates_data_new_1,"X")
            push!(Gates_data_new_2,0.0)
            push!(Gates_data_new_3,Gates_data_3[i]) 
        
            push!(U_noise_list,x_matrix) # Noise.
        
            push!(U_list,Matrix_Gate(CX(0.0),Gates_data_3[i])) # Noiseless.
            
        else
            #push!(u0_list,"CRX")
        
            epsilon = NOISE[i]
            push!(NOISE_list,epsilon)        
            rx_matrix = CU(Rx(Gates_data_1[i]+DELTA*epsilon), Gates_data_2[i], Gates_data_3[i])
            U_0_delta *= rx_matrix
        
            push!(Gates_data_new_1,Gates_data_1[i])
            push!(Gates_data_new_2,Gates_data_2[i])
            push!(Gates_data_new_3,Gates_data_3[i])
        
            push!(U_noise_list,rx_matrix) # Noise.
        
            push!(U_list,CU(Rx(Gates_data_1[i]), Gates_data_2[i], Gates_data_3[i])) # Noiseless.
            
        end
    end
        
    GROVER_DELTA = U_x_delta*U_0_delta
    
    function kth_term(k)

            f_k = Identity(2^L);
    
            for i = k:length(U_list)-1
                f_k = f_k*collect(U_list[length(U_list)-i+k])
            end     
            #= Corresponding H for the kth term. =#
            if Gates_data_new_1[k] == "H"

                Qubit = Gates_data_new_3[k] # qubit.
                H_k = Matrix_Gate(I2-H,Qubit) #= H_had = I2-Had. =#

            elseif Gates_data_new_1[k] == "X"

                Qubit = Gates_data_new_3[k] # qubit.
                H_k = Matrix_Gate([1 1;1 1],Qubit) #= H_X = X+I2. =#

            else
        
                Angle = Gates_data_new_1[k]
                Control_Qubit = int(Gates_data_new_2[k])
                Target_Qubit = int(Gates_data_new_3[k])
                #= H = ((I-Z)/2)_c \otimes ((I+X)/2)_t.=#
                Matrices = Dict("I" => I,"U" => [1 1;1 1]/2, "PI_1" => (I2-Z)/2)
                p1 = fill("I", L)
                p1[Control_Qubit] = "PI_1"
                p1[Target_Qubit] = "U"
                H_k = Matrices[p1[1]]
                for i = 2:L
                    H_k = kron(H_k,Matrices[p1[i]])
                end                                 
            end
    
    
        return f_k*H_k*(f_k')
    end;        
    
    EIGU = py"eigu"(collect(GROVER_DELTA))
    E_exact = real(1im*log.(EIGU[1])); # Eigenvalue.
    E_exact = E_exact[2:2^L-1]; #= Neglecting the two special states at 1 and 2^L. =#
    
    #= The following loop sums over all epsilon to get H_eff. =#
    h_eff = zeros(2^L,2^L);
    for i = 1:length(U_list)
        h_eff += NOISE_list[i]*kth_term(i)
    end        

    h_eff = DELTA * h_eff # Matrix in Z basis.
    h_eff_D = (V')*h_eff*(V) # Matrix in |0> and |xbar> basis.
    h_eff_D = exp(-1im*h_eff_D[3:2^L,3:2^L]) # |0> and |xbar> basis states are deleted.
    E_eff_D = py"eigu"(h_eff_D)[1] # Matrix is diagonalized.
    E_eff_D = real(1im*log.(E_eff_D)) # Extracing phi_f from exp(-i*phi_F).
    E_eff_D_sorted = sort(real(E_eff_D),rev = true); # Soring the eigenvalues in descending order.

    
    return E_exact, E_eff_D_sorted
    #return GROVER_DELTA
end;

In [171]:
#E_exact

In [172]:
#E_eff

In [173]:
#=G_noise = Identity(2^L)
for U in U_noise_list
    G_noise = G_noise*U
end
G_noise=#

### Comparison of approximation

In [174]:
#=
t_1 = U_list[1]*(Identity(2^L)-1im*DELTA*NOISE_list[1]*Matrix_Gate(I2-H,1))#-collect(U_noise_list[1])
t_2 = U_list[2]*(Identity(2^L)-1im*DELTA*NOISE_list[2]*Matrix_Gate([1 1;1 1],1))#-collect(U_noise_list[2])
t_3 = U_list[3]*(Identity(2^L)-1im*DELTA*NOISE_list[3]*kron((I2-Z)/2,[1 1;1 1]/2))#-collect(U_noise_list[3])
t_4 = U_list[4]*(Identity(2^L)-1im*DELTA*NOISE_list[4]*Matrix_Gate([1 1;1 1],1))#-collect(U_noise_list[4])
t_5 = U_list[5]*(Identity(2^L)-1im*DELTA*NOISE_list[5]*Matrix_Gate(I2-H,1))#-collect(U_noise_list[5])
t_6 = U_list[6]*(Identity(2^L)-1im*DELTA*NOISE_list[6]*Matrix_Gate([1 1;1 1],1))#-collect(U_noise_list[6])
t_7 = U_list[7]*(Identity(2^L)-1im*DELTA*NOISE_list[7]*Matrix_Gate(I2-H,2))#-collect(U_noise_list[7])
t_8 = U_list[8]*(Identity(2^L)-1im*DELTA*NOISE_list[8]*kron((I2-Z)/2,[1 1;1 1]/2))#-collect(U_noise_list[8])
t_9 = U_list[9]*(Identity(2^L)-1im*DELTA*NOISE_list[9]*Matrix_Gate([1 1;1 1],1))#-collect(U_noise_list[9])
t_10 = U_list[10]*(Identity(2^L)-1im*DELTA*NOISE_list[10]*Matrix_Gate(I2-H,2))#-collect(U_noise_list[10])=#

In [175]:
#U_x = t_1*t_2*t_3*t_4*t_5

In [176]:
#U_0 = t_6*t_7*t_8*t_9*t_10

In [177]:
#=H1 = Matrix_Gate(I2-H,1)
H2 = Matrix_Gate([1 1;1 1],1)
H3 = kron((I2-Z)/2,[1 1;1 1]/2)
H4 = Matrix_Gate([1 1;1 1],1)
H5 = Matrix_Gate(I2-H,1)
H6 = Matrix_Gate([1 1;1 1],1)
H7 = Matrix_Gate(I2-H,2)
H8 = kron((I2-Z)/2,[1 1;1 1]/2)
H9 = Matrix_Gate([1 1;1 1],1)
H10 = Matrix_Gate(I2-H,2)=#

In [178]:
#=H_eff_10 = NOISE_list[10]*H10
H_eff_9 = NOISE_list[9]*U_list[10]*H9*(U_list[10])'
H_eff_8 = NOISE_list[8]*U_list[10]*U_list[9]*H8*(U_list[10]*U_list[9])'
H_eff_7 = NOISE_list[7]*U_list[10]*U_list[9]*U_list[8]*H7*(U_list[10]*U_list[9]*U_list[8])'
H_eff_6 = NOISE_list[6]*U_list[10]*U_list[9]*U_list[8]*U_list[7]*H6*(U_list[10]*U_list[9]*U_list[8]*U_list[7])'
H_eff_5 = NOISE_list[5]*U_list[10]*U_list[9]*U_list[8]*U_list[7]*U_list[6]*H5*(U_list[10]*U_list[9]*U_list[8]*U_list[7]*U_list[6])'
H_eff_4 = NOISE_list[4]*U_list[10]*U_list[9]*U_list[8]*U_list[7]*U_list[6]*U_list[5]*H4*(U_list[10]*U_list[9]*U_list[8]*U_list[7]*U_list[6]*U_list[5])'
H_eff_3 = NOISE_list[3]*U_list[10]*U_list[9]*U_list[8]*U_list[7]*U_list[6]*U_list[5]*U_list[4]*H3*(U_list[10]*U_list[9]*U_list[8]*U_list[7]*U_list[6]*U_list[5]*U_list[4])'
H_eff_2 = NOISE_list[2]*U_list[10]*U_list[9]*U_list[8]*U_list[7]*U_list[6]*U_list[5]*U_list[4]*U_list[3]*H2*(U_list[10]*U_list[9]*U_list[8]*U_list[7]*U_list[6]*U_list[5]*U_list[4]*U_list[3])'
H_eff_1 = NOISE_list[1]*U_list[10]*U_list[9]*U_list[8]*U_list[7]*U_list[6]*U_list[5]*U_list[4]*U_list[3]*U_list[2]*H1*(U_list[10]*U_list[9]*U_list[8]*U_list[7]*U_list[6]*U_list[5]*U_list[4]*U_list[3]*U_list[2])'=#

In [179]:
#h_eff_manual = DELTA*(H_eff_1+H_eff_2+H_eff_3+H_eff_4+H_eff_5+H_eff_6+H_eff_7+H_eff_8+H_eff_9+H_eff_10)

In [180]:
#V = py"eigu"(G_exact)[2]
#round.(V'*V,digits=2)

In [181]:
#h_eff_D = V*h_eff_manual*V'
#h_eff_D = h_eff_D[3:2^L,3:2^L]
#E_eff_D = eigen(h_eff_D).values
#sort(real(E_eff_D),rev = true)

In [182]:
#E_exact

In [183]:
function x_bar(n)
    k(n) = ((2*pi*n)/(2^L-2)) # n = 0,1,...,2^L-2 .
    function ket(j) # j = 0,1,2,...,2^L-1 .x
        my_ket = zeros(2^L)
        my_ket[j+1] = 1
        return my_ket
    end
    s = zeros(2^L)
    for j = 1:2^L-2
        s += exp(1im*j*k(n)) .* ket(j)
    end
    return s/sqrt(2^L-2)
end;

In [184]:
#=
# 10 th term. H_2
H10 = Matrix_Gate(I2-H,2)
# 9 th term. X_1
H9 = U_list[10]*Matrix_Gate([1 1;1 1],1)*(U_list[10])'
# 8 th term. CX_1_2
M8 = kron((I-Z)/2,[1 1;1 1]/2)
H8 = U_list[10]*U_list[9]*M8*(U_list[10]*U_list[9])'
# 7 th term. H_2
H7 = U_list[10]*U_list[9]*U_list[8]*Matrix_Gate(I2-H,2)*(U_list[10]*U_list[9]*U_list[8])'
# 6 th term. X_1
H6 = U_list[10]*U_list[9]*U_list[8]*U_list[7]*Matrix_Gate([1 1;1 1],1)*(U_list[10]*U_list[9]*U_list[8]*U_list[7])'
# 5 th term. H_1
H5 = U_list[10]*U_list[9]*U_list[8]*U_list[7]*U_list[6]*Matrix_Gate(I2-H,1)*(U_list[10]*U_list[9]*U_list[8]*U_list[7]*U_list[6])'
# 4 th term. X_1
H4 = U_list[10]*U_list[9]*U_list[8]*U_list[7]*U_list[6]*U_list[5]*Matrix_Gate([1 1;1 1],1)*(U_list[10]*U_list[9]*U_list[8]*U_list[7]*U_list[6]*U_list[5])'
# 3 rd term. CX_12
M3 = kron((I-Z)/2,[1 1;1 1]/2)
H3 = U_list[10]*U_list[9]*U_list[8]*U_list[7]*U_list[6]*U_list[5]*U_list[4]*M3*(U_list[10]*U_list[9]*U_list[8]*U_list[7]*U_list[6]*U_list[5]*U_list[4])'
# 2 nd term. X_1
H2 = U_list[10]*U_list[9]*U_list[8]*U_list[7]*U_list[6]*U_list[5]*U_list[4]*U_list[3]*Matrix_Gate([1 1;1 1],1)*(U_list[10]*U_list[9]*U_list[8]*U_list[7]*U_list[6]*U_list[5]*U_list[4]*U_list[3])'
# 1 st term. H_1
H1 = U_list[10]*U_list[9]*U_list[8]*U_list[7]*U_list[6]*U_list[5]*U_list[4]*U_list[3]*U_list[2]*Matrix_Gate(I2-H,1)*(U_list[10]*U_list[9]*U_list[8]*U_list[7]*U_list[6]*U_list[5]*U_list[4]*U_list[3]*U_list[2])'=#

In [185]:
#H_EFF = NOISE_list[10]*H10+NOISE_list[9]*H9+NOISE_list[8]*H8+NOISE_list[7]*H7+NOISE_list[6]*H6+NOISE_list[5]*H5+NOISE_list[4]*H4+NOISE_list[3]*H3+NOISE_list[2]*H2+NOISE_list[1]*H1

In [186]:
#eigen(DELTA*H_EFF[3:2^L,3:2^L]).values

### Grover matrix from calculating the Hs

In [187]:
#for i in U_noise_list
    #println(i[1])
#end

In [188]:
#Gates_data_new_1

In [189]:
#Gates_data_new_2

In [190]:
#Gates_data_new_3

In [191]:
#collect(U_noise_list[4])
#real(U_list[2])
#X = exp(-1im*(pi/2)*(I2-[0 1;1 0]));
#real(Matrix_Gate(X,1))

In [192]:
#U_list[1]*(Identity(2^L)-1im*DELTA*NOISE[1]*Matrix_Gate(I2-H,1))
#collect(X*(Identity(2^L)-1im*DELTA*NOISE_list[4]*Matrix_Gate([1 1;1 1],1)))

### Exact
$X\otimes I_{2} = \exp[-i (\frac{\pi}{2}+\delta \epsilon)\{(I_{2}-X)\otimes I_{2}\} ]$

In [193]:
#delta = 0.08
#X_gate = [0 1;1 0]
#U_EXACT = exp(-1im*(pi/2+delta*0.1)*kron([1 0;0 1]-X_gate,[1 0;0 1]))[1,1]

### Approximation
$X\otimes I_{2} = \exp[-i(\frac{\pi}{2})(I_{2}-X)\otimes I_{2}]\{I_{4}-i\delta\epsilon(I_{2}-X)\otimes I_{2}\} = (X \otimes I_{2})\{1-i\delta\epsilon(I_{2}-X)\otimes I_{2}\}$

In [194]:
#U_APPROX = exp(-1im*(pi/2+0.01*0.1)*kron([1 0;0 1]-X_gate,[1 0;0 1])) * ([1 0 0 0;0 1 0 0;0 0 1 0;0 0 0 1]-1im*delta*0.1*kron([1 0;0 1]-X_gate,[1 0;0 1]))

In [195]:
#det(U_EXACT.-U_APPROX)

In [196]:
py"""
f = open('new_eigenvalues_data'+'.txt', 'w')
def Write_file2(delta, effective, exact):
    f = open('new_eigenvalues_data'+'.txt', 'a')
    f.write(str(delta) + '\t' + str(effective)+ '\t' + str(exact) +'\n')
"""
Num = 100;
for i = 1:Num
    delta = 0.3*(i/Num)

    EE = Eigenvalues(delta)
    Exact = EE[1]
    Effec = EE[2]
    #println(Exact)
    #println(Effec)    
    for j = 1:2^L-2
        py"Write_file2"(delta,Exact[j],Effec[j])

        #println(delta);
    end
end